Transpose / Pivot table Pandas

In [ ]:
import pandas as pd

df.set_index('Material Name', inplace=True)

trans = df.transpose()

trans.reset_index(inplace=True)

trans = trans[cols]

# One way to do it, for now it can not remain same order easily.
trans.melt(id_vars=["index"], ignore_index = False).to_excel('../output/hope.xlsx')

# Another way but preverse order.
result= trans.set_index(["index"]).stack()
# Or
result = df.pivot(index=["some cols"], columns="cols contain values to set name of columns", values="cols that want to get values from")

Convert to datetime

In [ ]:
df['columns name'] = pd.to_datetime(df['columns name']], format="%d/%m/%Y", yearfirst=True)

Dataframe to SQL

In [ ]:
from sqlalchemy import create_engine

df = pd.read_excel('Byun/take_in.xlsx', header=10)

sql_engine = create_engine(
        'mysql+pymysql://admin:password@ip/scheme', pool_recycle=3600
    )
take_in.to_sql('table', sql_engine,
              if_exists='replace')

Walk through files in folder

In [ ]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/bom'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)
dfs = [pd.read_excel(x, header=0, usecols=['Unnamed: 0', 'Plant', 'Entry Date', 'Material', 'Location',
       'Stock Type', 'Movement Type', 'Posting Date', 'Unit', 'Quantity',
       'Material Document No', 'Provide', 'Reference', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Previous Stock',
       'Entry User']) for x in file_names]
def loc_df(df):
    lst = [x[0] for x in df['Movement Type'].str.split()]
    df['movement type check'] = lst
    df = df.loc[df['movement type check'].isin(['101','102','103'])]
    return df

dfs = [loc_df(df) for df in dfs]

result = pd.concat(dfs)
result.columns
result = result[['Material', 'Unit', 'Movement Type', 'Quantity', 'Reference']]
end = result.pivot_table(index=['Material', 'Unit'], columns=['Reference'], aggfunc=np.sum)
end.to_excel('../output/GIANGALO.xlsx')



Concat all sheets in file

In [ ]:
import pandas as pd
import numpy as np
dfs = pd.read_excel('../data/huy_data/TK 1571-2021 NGUYET GOI CHI THUY (1).xlsx', sheet_name=None, header=5,
    #    usecols=['TT', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'USD',
    #    'Số lượng', 'Số lượng.1', 'Số lượng.2', 'Số lượng.3']
)
# for name, df in dfs.items():
#     print(df.columns)

# def loc_df(df):
#     df = df.loc[~(pd.isna(df['Unnamed: 3']))]
#     return df
dfs = list(dfs.values())
# dfs = [loc_df(df) for df in dfs]
result = pd.concat(dfs)
result.to_excel('../output/TK 1571-2021 NGUYET GOI CHI THUY.xlsx')


Concat BOM in folder

In [ ]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/huy_data/HĐ 2021'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)

dfs = [pd.read_excel(x, header=10
       ) for x in file_names]
# for x in dfs:
#     print(x.columns)

def loc_df(df):
    df = df.loc[~pd.isna(df['(5)'])]
    return df

dfs = [loc_df(df) for df in dfs]
result = pd.concat(dfs)
result.ffill(inplace=True)
# result.to_excel('../output/HĐ 2021.xlsx')


In [ ]:
result.set_index(['(1)','(2)', '(3)','(4)']).stack().to_excel('../output/HĐ 2021.xlsx')

In [ ]:
import pandas as pd
import numpy as np
from os import walk

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 40)
path = '../data/GIANG'
file_names = next(walk(path), (None, None, []))[2]
file_names = (f'{path}/{x}' for x in file_names)
dfs = [pd.read_excel(x, header=1, 
            # usecols=['Unnamed: 0', 'Plant', 'Entry Date', 'Material', 'Location',
            # 'Stock Type', 'Movement Type', 'Posting Date', 'Unit', 'Quantity',
            # 'Material Document No', 'Provide', 'Reference', 'Unnamed: 13',
            # 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Previous Stock',
            # 'Entry User']
            ) 
            for x in file_names
       ]
# def loc_df(df):
#     lst = [x[0] for x in df['Movement Type'].str.split()]
#     df['movement type check'] = lst
#     df = df.loc[df['movement type check'].isin(['101','102','103'])]
#     return df

# dfs = [loc_df(df) for df in dfs]

result = pd.concat(dfs)
result.to_excel('../output/GIANGALO.xlsx')



In [ ]:
import pandas as pd
from pandas import ExcelWriter


# df = pd.read_excel('../data/COSTING BOM T1-12.xlsx', header=5)

dfs = pd.read_excel('../data/COSTING BOM T1-12.xlsx', sheet_name=None, header=5,)

def loc_df(df):
    columns = df.columns
    columns = [column for column in columns if not str(column).startswith('Unnamed')]
    df = df[columns]
    df = df.loc[~pd.isna(df['Mã']) & ~(df['Mã'] == 'Đơn giá NVL') & ~(df['Mã'] == 0)]
    result= df.set_index(["Mã"]).stack()

    result = pd.DataFrame(result)
    result.reset_index(inplace=True)
    result = result.drop(result[result[0] == 0].index)
    return result

dfs = [loc_df(df) for df in dfs.values()]

result = pd.concat(dfs)

with ExcelWriter('../output/end.xlsx') as writer:
    for n, df in enumerate(dfs, start=1):
        df.rename(columns={
            'Mã': 'TP',
            'level_1': 'NVL',
            0: 'VALUE'
        }, inplace=True)
        df.to_excel(writer,'sheet %s' % n, index=False)


GIANG

In [ ]:
import pandas as pd

df = pd.read_excel('../data/BOM INZI.xlsb')

df = df.loc[~pd.isna(df['Semi']) & ~pd.isna(df['Technical BOM'])]
df = df[['Level.', 'Product', 'Semi', 'RM', 'Name', 'Unit','Technical BOM']]
df['Level.'] = [x.split('.')[-1] for x in df['Level.']]
df['Level.'] = df['Level.'].astype(int)

# rats = []
# previous_level = 1
# ratio = [1] * (len(df['Level.'].unique()) + 1)

# for row in zip(df['Level.'], df['Technical BOM']):
#     if row[0] > 1:
#         if row[0] > previous_level:
#             ratio[row[0]] = ratio[row[0] - 1] * previous_ratio
    
#     rats.append(ratio[row[0]])

#     previous_ratio = row[1]
#     previous_level = row[0]

# df['ratio'] = rats
# df['BOM'] = df['Technical BOM'] * df['ratio']



indexs = []
current_level = 1
for count, x in enumerate(df['Level.'], start=-1):
    if x > current_level:
        indexs.append(count)
    current_level = x

df = df.drop(df.index[indexs])

df.to_excel('../output/ENDING.xlsx')



In [89]:
import pandas as pd

df = pd.read_excel('../data/BCQT2021.BOM.xlsx', header=9)
df = df.groupby(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Mã', 'Tên',
       'Đơn vị tính']).sum()
df.to_excel('../output/bcqt.xlsx')


In [5]:
import pandas as pd

df = pd.read_excel('../data/(03.04.2022)_MEGAELEC_16 FORM_PROCESSING.xlsb', header=10)
df = df.groupby(['(1)', '(2)', '(3)', '(4)', '(5)', '(6)', '(7)']).sum()
df.reset_index().to_excel('../output/GIANGx.xlsx')
print(df.shape[0])

93352


In [3]:
import pandas as pd

def Insert_row(row_number, df, row_value):
    # Starting value of upper half
    start_upper = 0
  
    # End value of upper half
    end_upper = row_number
  
    # Start value of lower half
    start_lower = row_number
  
    # End value of lower half
    end_lower = df.shape[0]
  
    # Create a list of upper_half index
    upper_half = [*range(start_upper, end_upper, 1)]
  
    # Create a list of lower_half index
    lower_half = [*range(start_lower, end_lower, 1)]
  
    # Increment the value of lower half by 1
    lower_half = [x.__add__(1) for x in lower_half]
  
    # Combine the two lists
    index_ = upper_half + lower_half
  
    # Update the index of the dataframe
    df.index = index_
  
    # Insert a row at the end
    df.loc[row_number] = row_value
   
    # Sort the index labels
    df = df.sort_index()
  
    # return the dataframe
    return df
  


df = pd.read_excel('../data/(01.04.2022)_MEGAELEC_16 FORM_PROCESSING copy (1).xlsb', header=11)
k = 0

df = df.loc[df['(5)'] != 'EX-04-00017']
current = '00-000-2276-REV-E-MG'
for count, x in enumerate(df['(2)']):
    if x != current:
        for _ in range(11):
            df = Insert_row(count + k, df, pd.Series())
        k += 11
    current = x
    

df.to_excel('../output/aaaaa.xlsx')


C:\Users\Thinh\AppData\Local\Temp/ipykernel_4416/2036417805.py:50: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = Insert_row(count + k, df, pd.Series())
